<a href="https://colab.research.google.com/github/jahanvi513/Video-Captioning/blob/main/VLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers accelerate torch torchvision opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import cv2
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
from torchvision import transforms
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl").to(device)

def extract_keyframes(video_path, num_frames=3):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(1, frame_count // num_frames)
    frames = []

    for i in range(num_frames):
        frame_num = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        if ret:
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frames.append(image)

    cap.release()
    return frames

def caption_frame(image):
    inputs = processor(images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, max_new_tokens=50)
    caption = processor.decode(generated_ids[0], skip_special_tokens=True)
    return caption

def caption_video(video_path):
    print(f"\n[INFO] Processing: {os.path.basename(video_path)}")
    frames = extract_keyframes(video_path)
    captions = [caption_frame(frame) for frame in frames]

    print("\n[DEBUG] Frame Captions:")
    for i, cap in enumerate(captions):
        print(f"  Frame {i+1}: {cap}")

    final_summary = " ".join(captions)
    return final_summary

video_folder = "/content/video-data"
video_files = [f for f in os.listdir(video_folder) if f.endswith(('.avi', '.mp4'))][:10]

results = {}
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    description = caption_video(video_path)
    results[video_file] = description

print("\n=== Video Descriptions ===")
for video, desc in results.items():
    print(f"\n{video}:\n{desc}")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/5.81G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]


[INFO] Processing: SoccerPenalty.avi

[DEBUG] Frame Captions:
  Frame 1: a picture of a soccer field with a ball in it
  Frame 2: a picture of a field with a lot of people
  Frame 3: a picture of a man playing a game of soccer

[INFO] Processing: ApplyEyeMakeup.avi

[DEBUG] Frame Captions:
  Frame 1: a woman with a black hair and a yellow background
  Frame 2: a woman is holding a pink lipstick
  Frame 3: a woman's face with a mirror in the background

[INFO] Processing: WalkingWithDog.avi

[DEBUG] Frame Captions:
  Frame 1: a man is riding a bike on a track
  Frame 2: a man is walking down the street with a bike
  Frame 3: a picture of a man in a blue shirt

[INFO] Processing: YoYo.avi

[DEBUG] Frame Captions:
  Frame 1: a man is holding a cigarette in his hand
  Frame 2: a man is standing in front of a tree
  Frame 3: a man is standing in front of a building

[INFO] Processing: UnevenBars.avi

[DEBUG] Frame Captions:
  Frame 1: a black screen with a train on it
  Frame 2: a train is